In [1]:
import os
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

from typing import Union

import numpy as np
import pandas as pd

import transformers
from sklearn.preprocessing import LabelEncoder

from modelling.models import TextProductMatch
from sklearn.model_selection import KFold
from transformers import BertTokenizer, TFBertModel

import os

params = {
    "N_CLASSES": 11014,
    "MAX_LEN": 20,
    "MODEL_NAME": 'bert-base-multilingual-uncased',
    "POOLING": "global_avg_1d",
    "EPOCHS": 5,
    "BATCH_SIZE": 16,
    "METRIC": "adacos"
}
PATH_NAME = 'saved/arcface/v1'
os.makedirs(PATH_NAME,exist_ok=True)

GPU found


In [2]:
from modelling.metrics import *
from modelling.pooling import *

In [3]:
config = transformers.BertConfig.from_pretrained(params["MODEL_NAME"])
config.output_hidden_states = True
word_model = transformers.TFAutoModel.from_pretrained(params["MODEL_NAME"],config=config)
tokenizer = transformers.AutoTokenizer.from_pretrained(params["MODEL_NAME"])

Some layers from the model checkpoint at bert-base-multilingual-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [5]:
params = {
    "N_CLASSES": 11014,
    "MAX_LEN": 70,
    "MODEL_NAME": 'bert-base-multilingual-uncased',
    "POOLING": "global_avg_1d",
    "EPOCHS": 5,
    "BATCH_SIZE": 32,
    "METRIC": "adacos",
    "LAST_HIDDEN_STATES": 3 
}

In [6]:
ids = tf.keras.layers.Input((params["MAX_LEN"],), dtype=tf.int32)
att = tf.keras.layers.Input((params["MAX_LEN"],), dtype=tf.int32)
tok = tf.keras.layers.Input((params["MAX_LEN"],), dtype=tf.int32)

labels_onehot = tf.keras.layers.Input(shape=(params["N_CLASSES"]), dtype=tf.int32)

x = word_model(ids, attention_mask=att, token_type_ids=tok)[-1]
x1 = tf.stack([x[-i-1] for i in range(params["LAST_HIDDEN_STATES"])])
x1_mean = tf.math.reduce_mean(x1, axis=0)
x1_max = tf.math.reduce_max(x1, axis=0)

x1 = tf.concat([x1_mean, x1_max],axis=-1)

x1 = TextProductMatch(params["N_CLASSES"],
                    params["POOLING"],
                    metric=params["METRIC"],
                    use_fc=True)([x1, labels_onehot])

model = tf.keras.Model(inputs=[[ids, att, tok], labels_onehot], outputs=[x1])

print(model.summary())

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None

<tf.Tensor 'ada_cos_3/cosine_similarity_3/MatMul:0' shape=(None, 1000) dtype=float32>

<tf.Tensor 'global_average_pooling1d/Mean:0' shape=(None, 1536) dtype=float32>